In [1]:
import numpy as np
import pandas as pd
import os
import sys
import numpy as np
import yaml
import cv2
from scipy.interpolate import interp1d
from bisect import bisect

In [2]:
sample = "office1-1"
inpath = os.path.join("/media/prakhar/BIG_BAG/Capstone/office", sample)

f1path = os.path.join(inpath,"fisheye1.txt")
f2path = os.path.join(inpath,"fisheye2.txt")

depthpath = os.path.join(inpath,"depth.txt")
gyropath = os.path.join(inpath,"t265_gyroscope.txt")
accpath = os.path.join(inpath,"t265_accelerometer.txt")

outpath = "/media/prakhar/BIG_BAG/Capstone/Loris_stereoi/"+sample
#associations file

sensorpath = "/media/prakhar/BIG_BAG/Capstone/office/"+sample+"/sensors.yaml"
matpath = "/media/prakhar/BIG_BAG/Capstone/office/"+sample+"/trans_matrix.yaml"

timestamp_dir = "/home/prakhar/CMU/Capstone/code/ORB_SLAM3/Examples/Stereo-Inertial/Loris_TimeStamps/"
timestamp_path = os.path.join(timestamp_dir, sample+".txt")

imu_dir = "/home/prakhar/CMU/Capstone/code/ORB_SLAM3/Examples/Stereo-Inertial/Loris_IMU/"
imu_path = os.path.join(imu_dir, sample+".txt")

if not os.path.exists(timestamp_dir):
    os.mkdir(timestamp_dir)
if not os.path.exists(imu_dir):
    os.mkdir(imu_dir)

In [3]:
sensor_data = cv2.FileStorage(sensorpath, cv2.FILE_STORAGE_READ)
mat_data = cv2.FileStorage(matpath, cv2.FILE_STORAGE_READ)

In [4]:
intr_acc = sensor_data.getNode("t265_accelerometer").getNode("imu_intrinsic").mat().reshape(3,4)
intr_gyro = sensor_data.getNode("t265_gyroscope").getNode("imu_intrinsic").mat().reshape(3,4)
print(intr_acc.shape, intr_gyro.shape)
print(intr_acc)
print(intr_gyro)



(3, 4) (3, 4)
[[ 1.01389265  0.          0.         -0.05109533]
 [ 0.          1.01744807  0.          0.2732152 ]
 [ 0.          0.          1.02856934  0.09496002]]
[[0.9956795  0.         0.         0.00192035]
 [0.         0.99803746 0.         0.00215618]
 [0.         0.         0.99269545 0.00119974]]


In [5]:
def parse_block(block):
    k1 = block[0].split(": ")[1]
    k2 = block[1].split(": ")[1]
    k3 = "".join(block[6:13]).replace(" ", "").split("data:")[1].replace("[","").replace("]","").split(",")
    k3 = np.array([float(l) for l in k3]).reshape((4,4))
    return k1,k2,k3

def get_trans_mats(matpath):
    with open(matpath,"r") as fp:
        data = [l[:-1] for l in fp.readlines()]
    # print(data)
    ls_blocks = []
    lsl = None
    for l in data:
        if '   -' == l:
            if lsl is not None:
                ls_blocks.append(lsl)
            lsl = []
        else:
            if lsl is not None:
                lsl.append(l)
    if lsl is not None:
        ls_blocks.append(lsl)
    ls_blocks = [parse_block(block) for block in ls_blocks]
    dict_trans = dict()
    for block in ls_blocks:
        k1,k2,k3 = block
        print(k1)
        dict_trans[k1] = dict()
    for block in ls_blocks:
        k1,k2,k3 = block
        dict_trans[k1][k2] = k3
    return dict_trans

In [25]:
dict_trans = get_trans_mats(matpath)
extr_acc = dict_trans['t265_fisheye1_optical_frame']['t265_accelerometer']
extr_gyro = dict_trans['t265_fisheye1_optical_frame']['t265_gyroscope']
extr_stereo = dict_trans['t265_fisheye1_optical_frame']['t265_fisheye2_optical_frame']

base_link
base_link
base_link
d400_color_optical_frame
d400_color_optical_frame
d400_color_optical_frame
t265_fisheye1_optical_frame
t265_fisheye1_optical_frame
t265_fisheye1_optical_frame


In [26]:
print(extr_acc)
print(extr_gyro)
print(extr_stereo)
assert np.allclose(extr_acc, extr_gyro)

[[-9.99926331e-01  4.93412758e-03  1.10899109e-02  1.06999986e-02]
 [-4.90392248e-03 -9.99984197e-01  2.74920074e-03  7.27595761e-12]
 [ 1.11033006e-02  2.69461415e-03  9.99934726e-01 -2.91038305e-11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[-9.99926331e-01  4.93412758e-03  1.10899109e-02  1.06999986e-02]
 [-4.90392248e-03 -9.99984197e-01  2.74920074e-03  7.27595761e-12]
 [ 1.11033006e-02  2.69461415e-03  9.99934726e-01 -2.91038305e-11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 9.99971034e-01 -3.08590787e-03  6.95767878e-03  6.39765114e-02]
 [ 3.11211612e-03  9.99988092e-01 -3.75913383e-03  1.48267136e-04]
 [-6.94599558e-03  3.78067804e-03  9.99968729e-01 -3.98468372e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [29]:
# extr_acc_inv = np.linalg.inv(extr_acc).flatten()
# print(list(extr_acc_inv))
# extr_stereo_inv = np.linalg.inv(extr_stereo).flatten()
# print(list(extr_stereo_inv))
print(list(extr_acc.flatten()))

print(list(extr_stereo.flatten()))


[-0.9999263314167739, 0.00493412757806944, 0.011089910930018767, 0.0106999985874, -0.004903922475695185, -0.9999841965949376, 0.0027492007438955665, 7.27595761418e-12, 0.011103300578872334, 0.002694614150708582, 0.9999347257550535, -2.91038304567e-11, 0.0, 0.0, 0.0, 1.0]
[0.9999710335197896, -0.003085907873266471, 0.006957678776835974, 0.0639765113592, 0.003112116118197612, 0.9999880917521713, -0.0037591338254934705, 0.000148267135955, -0.006945995582404035, 0.0037806780408845384, 0.9999687293205325, -0.000398468371714, 0.0, 0.0, 0.0, 1.0]


In [8]:
def get_rgb_data(rgbpath):
    with open(rgbpath, "r") as fp:
        data = fp.readlines()
        data = [l[:-1].split(" ") for l in data]
        data = [[float(l[0]), l[0], l[1]] for l in data]
    return data

def get_aligned_depth_data(depthpath):
    with open(depthpath, "r") as fp:
        data = fp.readlines()
        data = [l[:-1].split(" ") for l in data]
        data = [[float(l[0]), l[0], 'aligned_'+l[1]] for l in data]
    return data

def get_gyroscope_data(gyropath, intr_gyro):
    with open(gyropath, "r") as fp:
        data = fp.readlines()
        data = [l[:-1].split(" ") for l in data]
    head = data[0]
    data = data[1:]
    data = np.array([np.array([float(k) for k in l]) for l in data])
    datam = data[:,1:]
    data_scaled = np.transpose(intr_gyro[:3,:3]@datam.T) - intr_gyro[:,3]
    data[:,1:] = data_scaled
    return head, data
    
def get_accelerometer_data(accpath, intr_acc):
    with open(accpath, "r") as fp:
        data = fp.readlines()
        data = [l[:-1].split(" ") for l in data]
    head = data[0]
    data = data[1:]
    data = np.array([np.array([float(k) for k in l]) for l in data])
    datam = data[:,1:]
    data_scaled = np.transpose(intr_acc[:3,:3]@datam.T) - intr_acc[:,3]
    data[:,1:] = data_scaled
    return head, data


In [9]:
head, data_gyro = get_gyroscope_data(gyropath, intr_gyro)
#take every alternate row of gyro data
data_gyro = data_gyro[::2]
print(head, data_gyro.shape)
head, data_acc = get_accelerometer_data(accpath, intr_acc)
print(head, data_acc.shape)

['#Time', 'Gx', 'Gy', 'Gz'] (2694, 4)
['#Time', 'Ax', 'Ay', 'Az'] (1684, 4)


In [10]:
ind_max = bisect(data_gyro[:,0], data_acc[-1,0])
ind_min = bisect(data_gyro[:,0], data_acc[0,0])
data_gyro = data_gyro[ind_min:ind_max,:]

In [11]:
100/(data_gyro[100,0] - data_gyro[0,0])

99.77714721795611

In [12]:
#create interpolators
acc_interpolator = interp1d(data_acc[:,0], data_acc[:,1:], axis=0)
#interpolate at precise timesteps
acc_interp = acc_interpolator(data_gyro[:,0])

In [13]:
def write_IMU_readings(head, imu_mat, imu_path):
    with open(imu_path, "w") as fp:
        fp.write(head)
        for idr in range(imu_mat.shape[0]):
            reading = imu_mat[idr]
            ts = str(int(reading[0]))
            lsr = [str(x) for x in reading[1:]]
            lsr = [ts] + lsr
            line = ",".join(lsr) + "\n"
            fp.write(line)
    print("Done.")

#imu hed
head = "#timestamp [ns],w_RS_S_x [rad s^-1],w_RS_S_y [rad s^-1],w_RS_S_z [rad s^-1],a_RS_S_x [m s^-2],a_RS_S_y [m s^-2],a_RS_S_z [m s^-2]\n"
imu_mat = np.concatenate([data_gyro, acc_interp], axis=1)
imu_mat[:,0] *= 1e9

write_IMU_readings(head, imu_mat, imu_path)

imu_dir2 = os.path.join(outpath, "mav0/imu0/")

if not os.path.exists(imu_dir2):
    print("Creating dir: "+imu_dir2)
    os.makedirs(imu_dir2)

imu_path2 = os.path.join(imu_dir2, "data.csv")
write_IMU_readings(head, imu_mat, imu_path2)


Done.
Done.


In [14]:
print(imu_mat[:3])

[[ 1.56000008e+18 -2.10128470e-02 -1.38514508e-02  1.97280162e-03
  -1.88323085e-03  9.69241752e+00  2.30532925e-01]
 [ 1.56000008e+18 -1.67700702e-02 -1.70410699e-02 -1.19974476e-03
   1.02311355e-02  9.65594677e+00  4.39458360e-01]
 [ 1.56000008e+18 -1.25272925e-02 -1.91674817e-02  9.15286086e-04
   3.23711964e-02  9.58929379e+00  5.36479906e-01]]


In [15]:
data_f1 = get_rgb_data(f1path)
data_f2 = get_rgb_data(f2path)

In [17]:
cam_path1 = os.path.join(outpath, "mav0/cam0/data/")
if not os.path.exists(cam_path1):
    print("Creating dir: "+cam_path1)
    os.makedirs(cam_path1)
cam_path2 = os.path.join(outpath, "mav0/cam1/data/")
if not os.path.exists(cam_path2):
    print("Creating dir: "+cam_path2)
    os.makedirs(cam_path2)

In [20]:
with open(timestamp_path, "w") as fp:
    for idx,(line1, line2) in enumerate(zip(data_f1, data_f2)):
        assert line1[1] == line2[1], print("Not same!!: ", line1[1], line2[1])
        
        tsstr = str(int(line1[0]*1e9))
        
        fp.write(tsstr+"\n")
        fname = tsstr+".png"
        dst1 = os.path.join(cam_path1, fname)
        dst2 = os.path.join(cam_path2, fname)
        
        src1 = os.path.join(inpath, line1[2])
        src2 = os.path.join(inpath, line2[2])
        command1 = "ln -sf "+src1+" "+dst1
        command2 = "ln -sf "+src2+" "+dst2
#         print(command1)
#         print(command2)
#         break
        os.system(command1)
        os.system(command2)
        if (idx+1)%100 == 0:
            print(idx+1,len(data_f1))

100 810
200 810
300 810
400 810
500 810
600 810
700 810
800 810


In [ ]:
"/media/prakhar/BIG_BAG/Capstone/office/office1-1/fisheye1/1560000083.949067.png"
"/media/prakhar/BIG_BAG/Capstone/office/office1-1/fisheye2/1560000083.949067.png"